In [ ]:
import os
hf_token = "HUGGINGFACE TOKEN HERE"
os.environ["HUGGINGFACEHUB_API_TOKEN"]=hf_token
os.environ["HF_TOKEN"]=hf_token
os.environ['HF_HOME'] = 'YOUR_HOME_DIR/.cache/huggingface/'
os.environ['TRANSFOMERS_CACHE'] = 'YOUR_HOME_DIR/.cache/huggingface/'
import transformers
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

In [ ]:
#llm = Ollama(model="llama31-8b-noCOT", request_timeout=300.0)

EMBED_MODEL = HuggingFaceEmbedding(model_name="abhinand/MedEmbed-large-v0.1")

#Settings.llm = llm
Settings.embed_model = EMBED_MODEL
embed_dim = len(EMBED_MODEL.get_text_embedding("hi"))

In [ ]:
## LOAD VECTORE INDEX FROM DISK (NOT WORKING PROPERLY)

from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from docling.document_converter import DocumentConverter
from llama_index.node_parser.docling import DoclingNodeParser
from docling.chunking import HybridChunker
from pathlib import Path

persist_dir = "YOUR_HOME_DIR/datasets/persistent_vector_store"

vector_store = MilvusVectorStore(
    uri=str(Path("YOUR_HOME_DIR/datasets/docling_md_vectordb.db")),
    dim=embed_dim,
    overwrite=False,
)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir=persist_dir),
    graph_store=SimpleGraphStore.from_persist_dir(persist_dir=persist_dir),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir=persist_dir),
)

index = load_index_from_storage(storage_context)

In [ ]:
## AD HOC CREATION OF VECTOR INDEX (WORKS BUT IS SLOWER)

from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from docling.document_converter import DocumentConverter
from llama_index.node_parser.docling import DoclingNodeParser
from docling.chunking import HybridChunker
from pathlib import Path

SOURCE = r"YOUR_HOME_DIR/guideline_edit.md"

reader = DoclingReader()
node_parser = MarkdownNodeParser()
chunker = HybridChunker()

vector_store = MilvusVectorStore(
    uri=str(Path("YOUR_HOME_DIR/datasets/docling_md_vectordb_infer.db")),
    dim=embed_dim,
    overwrite=True,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=reader.load_data(SOURCE),
    transformations=[node_parser],
    storage_context=storage_context,
    embed_model=EMBED_MODEL,
)

In [ ]:
import requests
question = "What is Pulmonary Atresia with Intact Ventricular Septum (PA-IVS) and what are its key morphological characteristics?"

retriever = index.as_retriever(similarity_top_k=5)
retrieved_docs = retriever.retrieve(question)
print(f"Retrieved {len(retrieved_docs)} docs")
sources = [s.get_content(s.metadata) for s in retrieved_docs]
sourcesStr = "\n\n".join(sources)

print("Sources:")
print(sourcesStr)
print("-" * 50)

QUERY = f"### Input:\n{question}\nContext:\n{sourcesStr}\n"

data = {
    "model": "qwen-32b-COT",
    "prompt": QUERY,
    "stream": False
}
url = "http://leinewr011.mh-hannover.local:11434/api/generate"
response = requests.post(url, json=data, timeout=60*60*1000)
text = response.json()["response"]
answer = text

print(f"\nAnswer: {answer}\n")